In [5]:
from scrape import svr_model
from scrape import xgb_model

In [6]:
results = xgb_model('te', scoring = 'HALF', csv_ = True)
results.head(50)

======= Fold 0 ========
Our accuracy on the validation set is 2.103314155574369
======= Fold 1 ========
Our accuracy on the validation set is 2.7497925455517462
======= Fold 2 ========
Our accuracy on the validation set is 2.6753145450743325
======= Fold 3 ========
Our accuracy on the validation set is 3.1289272029175033
======= Fold 4 ========
Our accuracy on the validation set is 2.774217868659549
iteration 1: Dimensions = (50, 2)
iteration 2: Dimensions = (100, 2)
iteration 3: Dimensions = (150, 2)
iteration 4: Dimensions = (200, 2)
iteration 5: Dimensions = (250, 2)


,name,proj fpts,class,recent rank
0,travis-kelce,11.235255,0,1
1,tj-hockenson,8.972775,0,2
2,mark-andrews,8.857463,0,4
3,george-kittle,8.840074,0,3
4,evan-engram,8.038818,0,5
5,dallas-goedert,7.886886,1,12
6,pat-freiermuth,7.835368,1,7
7,taysom-hill,7.272932,3,9
8,david-njoku,7.159383,1,11
9,cole-kmet,7.143723,2,8


In [7]:
results = svr_model('te', scoring = 'HALF', csv_ = True)
results.head(50)

PCA Selected
Train dimensions (pre PCA):  (223, 27)
Test dimensions (pre PCA):  (50, 27)
Train dimensions (post PCA):  (223, 11)
Test dimensions (post PCA):  (50, 11)
Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-3.511 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-4.017 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-3.285 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-3.420 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-3.705 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-3.214 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-3.788 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-3.127 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-3.322 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=0.1, kernel

,name,proj fpts,class,recent rank
0,travis-kelce,11.438303,0,1
1,mark-andrews,9.430119,0,4
2,tj-hockenson,9.177234,0,2
3,george-kittle,8.578910,0,3
4,evan-engram,8.465516,0,5
5,cole-kmet,7.506434,2,8
6,pat-freiermuth,7.485568,1,7
7,david-njoku,7.217340,1,11
8,dallas-goedert,7.155830,1,12
9,greg-dulcich,6.488479,2,29
